# NN-自然語言處理
## 教學目標
- 本教學著重於自然語言處理，其中涵蓋`MLP`、`RNN`以及`Transformers`。
- 這份教學的目標是介紹如何以 Python 和 PyTorch 實作神經網路。

## 適用對象
已經有基本的機器學習知識，且擁有 Python、`numpy`、`pandas`、`scikit-learn` 以及 `PyTorch` 基礎的學生。

若沒有先學過 Python，請參考 [python-入門語法](./python-入門語法.ipynb) 教學。

若沒有先學過 `pandas`，請參考 [pandas-基本功能](./pandas-基本功能.ipynb) 教學。

若沒有先學過 `numpy`，請參考 [numpy-基本功能](./numpy-基本功能.ipynb) 教學。

若沒有先學過 `scikit-learn`，請參考 [scikit-learn-基本功能](./scikit-learn-基本功能.ipynb) 教學。

若沒有先學過  `PyTorch` ，請參考 [PyTorch-基本功能](./PyTorch-基本功能.ipynb) 教學。

## 類神經網路 (NN)
- 類神經網路屬於一種機器學習方法
- 基本的神經網路模型由許多個非線性轉換的單元所構成 (如下圖所示)

![Imgur](https://i.imgur.com/tE4EdPh.png)
- 可參閱 [簡介](https://docs.google.com/presentation/d/1QeO_6cmset9AJB5qWWSaNwJbL8n2IePMc5t0vfzBBNA/edit#slide=id.p) 與 [實作](https://github.com/IKMLab/Feedforward-Tutorial/blob/master/text-classfication/text-classfication.ipynb) by IKMLab
- 神經網路又可以稱為多層感知機 (Perceptron)
- 深度學習則是模型透過深層神經網路來進行學習的方法 
- NN 的實作
    - 使用 TensorFlow/Keras/PyTorch 等深度學習函式庫
    - 也可以使用 scikit-learn 套件
        - `from sklearn.neural_network import MLPClassifier`
        - 但是要額外實作：
            1. 把 word 轉為 id
            2. embedding_lookup (或使用 torch.nn.embedding)
            3. 以 mini-batch 進行訓練 (fit 模型)

## 使用 NN 來進行中文的分類任務

- 我們將在這個教學裡讓大家實作中文情緒分析（Sentiment Analysis）
- 本資料集爲外賣平臺用戶評價分析，[下載連結](https://raw.githubusercontent.com/SophonPlus/ChineseNlpCorpus/master/datasets/waimai_10k/waimai_10k.csv)。
- 資料集欄位爲標籤（label）和評價（review），
- 標籤 1 爲正向，0 爲負向。
- 正向 4000 條，負向約 8000 條。

In [ ]:
!mkdir -p data
!wget https://raw.githubusercontent.com/SophonPlus/ChineseNlpCorpus/master/datasets/waimai_10k/waimai_10k.csv -O data/waimai_10k.csv
!pip install jieba
!pip install torch

In [ ]:
# 1. 導入所需套件

# Python 套件
import os
import math
import random

# 第3方套件
import jieba
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
# 2. 以 pandas 讀取資料
# 請先下載資料集

df = pd.read_csv("./data/waimai_10k.csv")

In [ ]:
# 3. 觀察資料

df.head()

## 建立字典
- 電腦無法僅透過字符來區分不同字之間的意涵
- 電腦視覺領域依賴的是影像資料本身的像素值
- 我們讓電腦理解文字的方法是透過向量
- 文字的意義藉由向量來進行表達的形式稱為 word embeddings
- 舉例:
$\textrm{apple}=[0.123, 0.456,0.789,\dots,0.111]$

- 如何建立每個文字所屬的向量？
    - 傳統方法: 計數法則
    - 近代方法 (2013-至今): 使用(淺層)神經網路訓練 word2vec ([參考](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/))，稱為 word embeddings
    - 現代方法 (2018-至今): 使用(深層)神經網路訓練 Transformers，也就是BERT ([參考](https://youtu.be/gh0hewYkjgo))，又稱為 contexualized embeddings
- 在那之前，要先建立分散式字詞的字典
    - 可粗分兩種斷詞方式 (tokenization):
        1. 每個字都斷 (character-level)
        2. 斷成字詞 (word-level)

## Word embeddings
- 著名的方法有:
    1. word2vec: Skip-gram, CBOW (continuous bag-of-words)
    2. GloVe
    3. fastText

In [ ]:
word_to_idx = {"好吃": 0, "棒": 1, "给力": 2}
embeds = torch.nn.Embedding(3, 5)  # 2 words in vocab, 5 dimensional embeddings

In [ ]:
lookup_tensor = torch.tensor(
    [
        word_to_idx["好吃"],
        word_to_idx["棒"],
        word_to_idx["给力"],
    ],
    dtype=torch.long,
)
word_embed = embeds(lookup_tensor)
print(word_embed)

### 分水嶺
1. 自己先建字典，透過模型中的 nn.embeddings 針對任務進行訓練 (本教學)
2. 自己先建字典，接著使用預訓練的 word embeddings 來初始化 nn.embeddings，然後針對任務進行訓練
    - 請參閱 [連結](https://colab.research.google.com/drive/13Fa0w7-AKtC0O06vCQHmOKAlPoV7PqOz?usp=sharing)
3. 不先建字典，直接針對任務的資料集預先訓練一個 word embeddings，接著使用預訓練的 word embeddings 來初始化 nn.embeddings，然後針對任務進行訓練
    - 請參閱 [連結](https://colab.research.google.com/drive/1DhNLBMnf5UwbF6xHYuxaa5VSCa51c4aS?usp=sharing)
4. 不先建字典，針對大規模通用資料集預先訓練一個 word embeddings，接著使用預訓練的 word embeddings 來初始化 nn.embeddings，然後針對任務進行訓練
    - 請參閱 [連結](http://zake7749.github.io/2016/08/28/word2vec-with-gensim/)，完成訓練後再至分水嶺2進行載入

In [ ]:
# 4. 設定隨機種子 (定義 function)
seed = 42

def set_seed(seed):
    """ 這個 function 可以使程式碼中有使用到 PyTorch 和 Numpy 的隨機過程受到控制
    Args:
        seed: 初始化 pseudorandom number generator 的正整數
    """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        #torch.cuda.manual_seed_all(seed)  # 如果有使用多個 GPU
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [ ]:
# 5. 建立字典
use_jieba=True

vocab = {'<pad>':0, '<unk>':1}

if use_jieba:
    words = []
    for sent in df['review']:
        tokens = jieba.lcut(sent, cut_all=False)
        words.extend(tokens)

else:
    # 以 character-level 斷詞
    words = df['review'].str.cat()

# 使字詞不重複
words = sorted(set(words))
for idx, word in enumerate(words):
    # 一開始已經放兩個進去 dictionary 了
    idx = idx + 2
    # 將 word to id 放到 dictionary
    vocab[word] = idx
        
# 查看字典大小
print("The vocab size is {}.".format(len(vocab)))

## 使用 PyTorch 建立 Dataset
![Imgur](https://i.imgur.com/wGnfCmH.png)

In [ ]:
# 6. 將資料分成 train/ validation/ test

train_data, test_data = train_test_split(
    df,
    test_size=0.2,
    random_state=seed
)
train_data, validation_data = train_test_split(
    train_data,
    test_size=0.1,
    random_state=seed
)

In [ ]:
# 7. 定義超參數

parameters = {
    "padding_idx": 0,
    "vocab_size": len(vocab),
    # Hyperparameters
    "embed_dim": 300,
    "hidden_dim": 256,
    "module_name": 'rnn', # 選項: rnn, lstm, gru, transformer
    "num_layers": 2,
    "learning_rate": 5e-4, # 使用 Transformer 時建議改成 5e-5
    "epochs": 10,
    "max_seq_len": 50,
    "batch_size": 64,
    # Transformers
    "nhead": 2,
    "dropout": 0.2,
}

In [ ]:
# 8. 建立 PyTorch Dataset (定義 class)

class WaimaiDataset(torch.utils.data.Dataset):
    # 繼承 torch.utils.data.Dataset
    def __init__(self, data, max_seq_len, use_jieba):
        self.df = data
        self.max_seq_len = max_seq_len
        # 可以選擇要不要使用結巴進行斷詞
        self.use_jieba = use_jieba
        
    # 改寫繼承的 __getitem__ function
    def __getitem__(self, idx):
        # dataframe 的第一個 column 是 label
        # dataframe 的第一個 column 是 評論的句子
        label, sent = self.df.iloc[idx, 0:2]
        # 先將 label 轉為 float32 以方便後面進行 loss function 的計算
        label_tensor = torch.tensor(label, dtype=torch.float32)
        if self.use_jieba:
            # 使用 lcut 可以 return list
            tokens = jieba.lcut(sent, cut_all=False)
        else:
            # 每個字都斷詞
            tokens = list(sent)
        
        # 控制最大的序列長度
        tokens = tokens[:self.max_seq_len]

        # 根據 vocab 轉換 word id
        # vocab 是一個 list
        tokens_id = [vocab[word] for word in tokens]
        tokens_tensor = torch.LongTensor(tokens_id)

        # 所以 第 0 個index是句子，第 1 個index是 label
        return tokens_tensor, label_tensor
    
    # 改寫繼承的 __len__ function
    def __len__(self):
        return len(self.df)

In [ ]:
# 9. 建立 PyTorch Dataset (執行 class)
use_jieba=use_jieba

trainset = WaimaiDataset(
    train_data,
    parameters["max_seq_len"],
    use_jieba=use_jieba
)
validset = WaimaiDataset(
    validation_data,
    parameters["max_seq_len"], 
    use_jieba=use_jieba
)
testset = WaimaiDataset(
    test_data, 
    parameters["max_seq_len"], 
    use_jieba=use_jieba
)

In [ ]:
# 10. 整理 batch 的資料 (定義 function)

def collate_batch(batch):
    # 抽每一個 batch 的第 0 個(注意順序)
    text = [i[0] for i in batch]
    # 進行 padding
    text = pad_sequence(text, batch_first=True)
    
    # 抽每一個 batch 的第 1 個(注意順序)
    label = [i[1] for i in batch]
    # 把每一個 batch 的答案疊成一個 tensor
    label = torch.stack(label)
    
    return text, label

In [ ]:
# 11. 建立資料分批 (mini-batches)

# 因為會針對 trainloader 進行 shuffle
# 所以在這個 cell 也執行一次 set_seed
# 對 trainloader 進行 shuffle 有助於降低 overfitting
set_seed(seed)

trainloader = DataLoader(
    trainset,
    batch_size=parameters["batch_size"],
    collate_fn=collate_batch,
    shuffle=True
)
validloader = DataLoader(
    validset,
    batch_size=parameters["batch_size"],
    collate_fn=collate_batch,
    shuffle=False
)
testloader = DataLoader(
    testset,
    batch_size=parameters["batch_size"],
    collate_fn=collate_batch,
    shuffle=False
)

## 建立模型
![Imgur](https://i.imgur.com/OgLBBm7.png)
- 模型建置的流程如上圖所示
- 文字的部份會透過 Dataset 及 DataLoader 進行處理
- embedding 層經由 nn.embedding 來實現 embedding lookup 的功能
- embedding 層再接上模型，最後接上分類層，即可進行分類任務
- 本範例提供的 Model class 可以藉由更換 module_name 來呼叫不同的 RNN

In [ ]:
# 12. 建立 RNN 模型 (定義 class)

class RNNModel(torch.nn.Module):
    def __init__(self, args):
        """定義能夠處理句子分類任務的 RNN 模型架構
        Arguments:
            - args (dict): 所需要的模型參數 (parameters)
        Returns:
            - None
        """
        super().__init__()
        # 模型參數
        self.padding_idx = args["padding_idx"]
        self.vocab_size = args["vocab_size"]
        self.embed_dim = args["embed_dim"]
        self.hidden_dim = args["hidden_dim"]
        self.module_name = args["module_name"]
        self.num_layers = args["num_layers"]
        self.dropout = args["dropout"]

        # 定義 Embedding 層
        self.embedding_layer = torch.nn.Embedding(
            num_embeddings=self.vocab_size,
            embedding_dim=self.embed_dim,
            padding_idx=self.padding_idx
        )
        # 定義 dropout 層
        self.embedding_dropout = torch.nn.Dropout(self.dropout)

        # 使用 RNN 系列的模型 (RNN/GRU/LSTM)
        module = self.get_hidden_layer_module()
        self.hidden_layer = module(
            input_size=self.embed_dim,
            hidden_size=self.hidden_dim,
            num_layers=self.num_layers,
            bidirectional=True,
            batch_first=True,
            dropout=self.dropout
        )
        # 將模型輸出到 output space
        self.output_layer = torch.nn.Linear(
            # 因為是 bi-directional，所以 self.hidden_dim*2
            in_features=self.hidden_dim*2,
            out_features=1
        )

    def get_hidden_layer_module(self):
        """根據指定的 module_name 回傳所使用的 PyTorch RNN 系列模型
        Arguments:
            - module_name (str): 模型名稱，選項為 rnn, gru, lstm
        Returns:
            - PyTorch 的模型模組 torch.nn.Module
        """
        if self.module_name == "rnn":
            return torch.nn.RNN
        elif self.module_name == "lstm":
            return torch.nn.LSTM
        elif self.module_name == "gru":
            return torch.nn.GRU
        raise ValueError("Invalid module name!")

    def forward(self, X):
        """定義神經網路的前向傳遞的進行流程
        Arguments:
            - X: 輸入值，維度為(B, S)，其中 B 為 batch size，S 為 sentence length
        Returns:
            - logits: 模型的輸出值，維度為(B, 1)，其中 B 為 batch size
            - Y: 模型的輸出值但經過非線性轉換 (這邊是用 sigmoid)，維度為(B, 1)，其中 B 為 batch size
        """
        # 維度: (B, S) -> (B, S, E)
        # B: batch size; S: sentence length; E: embedding dimension
        E = self.embedding_layer(X)
        E = self.embedding_dropout(E)
        
        # 使用 RNN 系列
        H_out, H_n = self.hidden_layer(E)

        # 取第一個和最後一個 hidden states做相加 (bi-directional)
        logits = self.output_layer(H_out[:, -1, :]+H_out[:, 0, :])
        Y = torch.sigmoid(logits)

        return logits, Y

## 使用 Transformer
![Imgur](https://i.imgur.com/58DPGG6.png)

In [ ]:
# 13. 建立 Transformer 模型 (定義 class)

class Transformer(torch.nn.Module):
    def __init__(self, args):
        """定義能夠處理句子分類任務的 Transformer encoder 模型架構
        Arguments:
            - args (dict): 所需要的模型參數 (parameters)        
        Returns:
            - None
        """
        super().__init__()
        # 模型參數
        self.padding_idx = args["padding_idx"]
        self.vocab_size = args["vocab_size"]
        self.embed_dim = args["embed_dim"]
        self.hidden_dim = args["hidden_dim"]
        self.num_layers = args["num_layers"]
        self.nhead = args["nhead"]
        self.dropout = args["dropout"]
        
        # 定義 Embedding 層
        self.embedding_layer = torch.nn.Embedding(
            num_embeddings=self.vocab_size,
            embedding_dim=self.embed_dim,
            padding_idx=self.padding_idx
        )
        # 定義 dropout 層
        self.embedding_dropout = torch.nn.Dropout(self.dropout)

        # 定義 Positional Encoding
        self.pos_encoder = PositionalEncoding(
            d_model=self.embed_dim,
            dropout=self.dropout
        )
        encoder_layer = TransformerEncoderLayer(
            d_model=self.embed_dim,
            nhead=self.nhead,
            dim_feedforward=self.hidden_dim,
            dropout=self.dropout
        )
        self.transformer_encoder = TransformerEncoder(
            encoder_layer=encoder_layer, 
            num_layers=self.num_layers
        )
        self.linear_layer = torch.nn.Linear(
            in_features=self.embed_dim,
            out_features=self.embed_dim
        )
        self.output_layer = torch.nn.Linear(
            in_features=self.embed_dim,
            out_features=1
        )

    def forward(self, X):
        """定義神經網路的前向傳遞的進行流程
        Arguments:
            - X: 輸入值，維度為(B, S)，其中 B 為 batch size，S 為 sentence length
        Returns:
            - logits: 模型的輸出值，維度為(B, 1)，其中 B 為 batch size
            - Y: 模型的輸出值但經過非線性轉換 (這邊是用 sigmoid)，維度為(B, 1)，其中 B 為 batch size
        """
        # 維度: (B * S) -> (B * S * E)
        # B: batch size; S: sentence length; E: embedding dimension
        E = self.embedding_layer(X)
        E = self.embedding_dropout(E)

        # 使用 Transformer
        # 輸出維度為 (B, S, E)
        E = self.pos_encoder(E)
        # 輸出維度為 (B, S, E)
        E = self.transformer_encoder(E)
        # 輸出維度為 (B, S, E)
        H_out = self.linear_layer(E)
        # 輸出維度為 (B, S, E)
        
        # 取第一個 hidden state
        logits = self.output_layer(H_out[:, 0, :])
        Y = torch.sigmoid(logits)
        
        return logits, Y

## Positional Encoding
- 功能: Transformer 使用 self-attention 機制中沒有考慮到序列順序，因此以 Positional Encoding 來加入順序資訊
- 數學公式如下所示

$PE_{pos, 2i}=sin(pos/10000^{2i/d_{model}})$

$PE_{pos, 2i+1}=cos(pos/10000^{2i/d_{model}})$

- 其中 d_model 是 embedding 的維度，i 是 embedding 的 index

In [ ]:
# 14. Positional Encoding (定義 class)

class PositionalEncoding(torch.nn.Module):
    r"""Inject some information about the relative or absolute position of the tokens
        in the sequence. The positional encodings have the same dimension as
        the embeddings, so that the two can be summed. Here, we use sine and cosine
        functions of different frequencies.

    Args:
        d_model: the embed dim (required).
        dropout: the dropout value (default=0.1).
        max_len: the max. length of the incoming sequence (default=5000).
    Examples:
        >>> pos_encoder = PositionalEncoding(d_model)
    """

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = torch.nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        r"""Inputs of forward function
        Args:
            x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        Examples:
            >>> output = pos_encoder(x)
        """

        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

## 設定訓練流程

In [ ]:
# 15. 設定訓練流程 (定義 function)

def train(trainloader, model, optimizer):
    """定義訓練時的進行流程
    Arguments:
        - trainloader: 具備 mini-batches 的 dataset，由 PyTorch DataLoader 所建立
        - model: 要進行訓練的模型
        - optimizer: 最佳化目標函數的演算法
    Returns:
        - train_loss: 模型在一個 epoch 的 training loss
    """
    # 設定模型的訓練模式
    model.train()
    
    # 記錄一個 epoch中 training 過程的 loss
    train_loss = 0
    # 從 trainloader 一次一次抽
    for x, y in trainloader:
        # 將變數丟到指定的裝置位置
        x = x.to(device)
        y = y.to(device)

        # 重新設定模型的梯度
        optimizer.zero_grad()
        
        # 1. 前向傳遞 (Forward Pass)
        logits, pred = model(x)

        # 2. 計算 loss (loss function 為二元交叉熵)
        loss_fn = torch.nn.BCELoss()
        loss = loss_fn(pred.squeeze(-1), y)

        # 3. 計算反向傳播的梯度
        loss.backward()
        # 4. "更新"模型的權重
        optimizer.step()

        # 一個 epoch 會抽很多次 batch，所以每個 batch 計算完都要加起來
        # .item() 在 PyTorch 中可以獲得該 tensor 的數值
        train_loss += loss.item()

    return train_loss

## 設定驗證流程

In [ ]:
# 16. 設定驗證流程 (定義 function)

def evaluate(dataloader, model):
    """定義驗證時的進行流程
    Arguments:
        - dataloader: 具備 mini-batches 的 dataset，由 PyTorch DataLoader 所建立
        - model: 要進行驗證的模型
    Returns:
        - loss: 模型在驗證/測試集的 loss
        - acc: 模型在驗證/測試集的正確率
    """
    # 設定模型的驗證模式
    # 此時 dropout 會自動關閉
    model.eval()
    
    # 設定現在不計算梯度
    with torch.no_grad():
        # 把每個 batch 的 label 儲存成一維 tensor
        y_true = torch.tensor([])
        y_pred = torch.tensor([])

        # 從 dataloader 一次一次抽
        for x, y in dataloader:
            # 把正確的 label concat 起來
            y_true = torch.cat([y_true, y])

            x = x.to(device)
            y = y.to(device)


            _, pred = model(x)
            # 預測的數值大於 0.5 則視為類別1，反之為類別0
            pred = (pred>0.5)*1
            # 把預測的 label concat 起來
            # 注意: 如果使用 gpu 計算的話，要先用 .cpu 把 tensor 轉回 cpu
            y_pred = torch.cat([y_pred, pred.cpu()])
            
    # 計算 loss (loss function 為二元交叉熵)
    loss_fn = torch.nn.BCELoss()
    # 模型輸出的維度是 (B, 1)，使用.squeeze(-1)可以讓維度變 (B,)
    loss = loss_fn(y_pred.squeeze(-1), y_true)
    # 計算正確率
    acc = accuracy_score(y_true, y_pred.squeeze(-1))
            
    return loss, acc

In [ ]:
# 17. 執行訓練所需要的準備工作

set_seed(seed)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

if parameters["module_name"] == 'transformer':
    model = Transformer(args=parameters)
else:
    model = RNNModel(args=parameters)

model = model.to(device)

# 使用 Adam 的演算法進行最佳化
opt = torch.optim.Adam(
    model.parameters(),
    lr=parameters["learning_rate"]
)

## 開始訓練

In [ ]:
# 18. 整個訓練及驗證過程的 script

train_loss_history = []
valid_loss_history = []

for epoch in range(parameters["epochs"]):
    train_loss = train(
        trainloader,
        model,
        optimizer=opt
    )
    
    print("Training loss at epoch {} is {}.".format(epoch+1, train_loss))
    train_loss_history.append(train_loss)
    
    if epoch % 2 == 1:
        print("=====Start validation=====")
        valid_loss, valid_acc = evaluate(
            dataloader=validloader,
            model=model
        )
        valid_loss_history.append(valid_loss)
        print("Validation accuracy at epoch {} is {}, and validation loss is {}."\
              .format(epoch+1, valid_acc, valid_loss))
        
    torch.save(model.state_dict(), "model_epoch_{}.pkl".format(epoch))

In [ ]:
# 19. 預測測試集

best_epoch = np.argmin(valid_loss_history)
model.load_state_dict(
    torch.load("model_epoch_{}.pkl".format(best_epoch))
)

print("=====Start testing=====")
test_loss, test_acc = evaluate(testloader, model)
print("Testing accuracy is {}.".format(test_acc))

## 任務完成，恭喜你學會了如何使用 `PyTorch` 來搭建神經網路
- 若想學習如何使用 RNN 來生成文字，請參考 [char-RNN-文本生成](./char-RNN-文本生成.ipynb) 教學。

## 什麼？你想使用 BERT？

In [ ]:
#!pip install simpletransformers

In [ ]:
# 載入所需的資料格式

train_bert = pd.DataFrame({
    "text": train_data["review"],
    "label": train_data["label"]
})
valid_bert = pd.DataFrame({
    "text": validation_data["review"],
    "label": validation_data["label"]
})
test_bert = list(test_data["review"])

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs


model_args = ClassificationArgs(sliding_window=True)

model = ClassificationModel(
    "bert",
    "bert-base-chinese",
    args=model_args,
)

In [ ]:
model.train_model(
    train_df=train_bert,
    eval_df=valid_bert
)

In [ ]:
result = model.predict(test_bert)

In [ ]:
accuracy_score(test_data['label'], result[0])